In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import time
from scipy.optimize import curve_fit,fminbound
import json
import switchbox
import chipcalibration.chipcalibration.punchout as po
from distproc.hwconfig import FPGAConfig, load_channel_configs
import qubitconfig.qchip as qc
from distproc import asmparse
import qubic.rpc_client as rc
import qubic.job_manager as jm
import qubic.state_disc as sd
import qubic.state_disc as sd
from IPython.display import clear_output

def run(qchip, ramp, fpga_config, channel_configs):
    reload(po)
    qubits = ['Q1']
    punchout = po.Punchout(qchip, fpga_config, channel_configs, qubits, n_freq=400, freq_bandwidth=6.e6,
                        atten_start=10, atten_stop=40, atten_step=1)
    punchout.run(runner)
    qchip = qc.QChip(qchippath+'/qubitcfg.json')
    punchout.run_punchout_gui(qchip,ramp)
chip='X4Y2'
board='sian'
print(board,chip)
total_duration = 10 * 60 * 60
interval = 10 * 60
start_time = time.time()
while time.time() - start_time < total_duration:
    ramps=['0.25','0.1','0.05']
    for ramp in ramps:
        print("On "+ramp)
        qchippath='/home/neelvora/readout/chipcalibration/submodules/qchip/X4Y2/sian/%s'%(ramp)
        fpga_config = FPGAConfig(**{'fpga_clk_period': 2.e-9, 'alu_instr_clks': 5, 
                                    'jump_cond_clks': 5, 'jump_fproc_clks': 5, 
                                    'pulse_regwrite_clks': 3, 'pulse_load_clks': 4})
        channel_configs = load_channel_configs(qchippath+'/channel_config.json')
        qchip = qc.QChip(qchippath+'/qubitcfg.json')
        reload(rc)
        runner = rc.CircuitRunnerClient(ip='192.168.1.25', port=9097)
        jobman = jm.JobManager(fpga_config, channel_configs, runner, qchip)
        run(qchip,ramp,fpga_config,channel_configs)
    clear_output(wait=True)
    print('sleeping')
    time.sleep(interval)
    clear_output(wait=True)
print("Loop completed after 10 hours.")
    